In [1]:
import torch
import numpy as np
import pandas as pd
import matplotlib
from matplotlib import pyplot as plt
import torchvision
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
import torchvision.transforms as transforms
from torchvision.transforms import ToTensor, Lambda, Compose, Normalize
from collections import defaultdict
from torch.utils.data import random_split
import copy

from NN_utils import *
from NN_utils.train_and_eval import *
from uncertainty import train_NN_with_g
from uncertainty.losses import penalized_uncertainty
import uncertainty.comparison as unc_comp
import uncertainty.quantifications as unc


from torch.utils.data import random_split

In [3]:
print(torch.cuda.is_available())
dev = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

True


In [11]:
transforms_train = transforms.Compose([
                    transforms.RandomCrop(32, padding=4),
                    transforms.RandomHorizontalFlip(),
                    transforms.ToTensor(),
                    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))])
transforms_ = transforms.Compose([
transforms.ToTensor(),
transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),])

In [14]:
training_data = datasets.CIFAR10(
root="data",
 train=True,
 download=True,
transform=transforms_train)

test_data = datasets.CIFAR10(
root="data",
train=False,
download=True,
transform=transforms_)

train_size = int(0.5*len(training_data))
val_size = len(training_data) - train_size
training_data, validation_data = random_split(training_data, [train_size, val_size])

Files already downloaded and verified
Files already downloaded and verified


In [16]:
batch_size = 12
train_dataloader = DataLoader(training_data, batch_size=batch_size,shuffle = True)
validation_dataloader = DataLoader(validation_data, batch_size=batch_size,shuffle = True)
test_dataloader = DataLoader(test_data, batch_size=100)

In [18]:
from NN_models.CIFAR10 import Model_CNN_with_g
model = Model_CNN_with_g()

In [21]:
def model_metrics(model,data):
    loss_criterion = nn.NLLLoss(reduction = 'sum')
    model.eval()
    dev = next(model.parameters()).device
    total = 0
    correct= 0
    g_i = 0
    bce = 0
    with torch.no_grad():
        for image,label in data:
            image,label = image.to(dev), label.to(dev)
            output = model(image)
            g = model.get_g()
            total += label.size(0)
            correct += correct_total(output,label)
            g_i += torch.sum(g).item()
            bce += loss_criterion(output,label).item()\

    return (correct/total),g_i/total, bce/total

In [19]:
class Trainer(nn.Module):
    def __init__(self,model,optimizer,loss_criterion, print_loss = True,keep_hist = True):
        # adaptar keep hist para definir oq manter\n",
        super().__init__()
        self.model = model
        self.optimizer = optimizer
        self.loss_fn = loss_criterion
        self.print_loss = print_loss
        self.keep_hist = keep_hist

        self.g_list = []
        self.acc_list = []
        self.bce_list = []

    def fit(self,data,n_epochs):
        for epoch in range(1,n_epochs+1):
            train_NN_with_g(self.model,self.optimizer,data,self.loss_fn,1, self.print_loss, set_train_mode = True)
            if self.keep_hist:
                self.update_hist(data)

    def fit_x(self,data,n_epochs,ignored_layers = ['fc_g_layer']):
        loss_criterion = copy.copy(self.loss_fn.criterion)
        loss_criterion.reduction = 'mean'
        ignore_layers(model,ignored_layers,reset = True)
        for epoch in range(1,n_epochs+1):
            train_NN(self.model,self.optimizer,data,loss_criterion,1, self.print_loss, set_train_mode = True)
            if self.keep_hist:
                self.update_hist(data)

    def fit_g(self,data,n_epochs,ignored_layers = ['conv_layer','fc_x_layer']):
        try:
            self.loss_fn.update_L0(self.bce_list[-1])
        except:
            self.loss_fn.update_L0(np.log(10))
            ignore_layers(model,ignored_layers, reset = True)
        for epoch in range(1,n_epochs+1):
            train_NN_with_g(self.model,self.optimizer,data,self.loss_fn,1, self.print_loss, set_train_mode = True)
            if self.keep_hist:
                self.update_hist(data)

            self.loss_fn.update_L0(self.bce_list[-1])

    def update_hist(self,data):
        acc, g, bce =  model_metrics(self.model,data)
        self.g_list.append(g)
        self.acc_list.append(acc)
        self.bce_list.append(bce)

In [ ]:
model = Model_CNN_with_g()
model = model.to(dev)
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

loss_criterion = nn.NLLLoss(reduction = 'none')
loss_fn = penalized_uncertainty(loss_criterion,np.log(10))

model_trainer = Trainer(model,optimizer,loss_fn, print_loss = True,keep_hist = True)
model_trainer.fit(train_dataloader,30)
acc, g, bce = model_metrics(model,train_dataloader)
print('Conjunto de treinamento: acc = ', acc, 'média de g = ', g, 'média de bce = ', bce, '\n')
acc, g, bce = model_metrics(model,test_dataloader)
print('Conjunto de teste: acc = ', acc, 'média de g = ', g, 'média de bce = ', bce, '\n')

Epoch  1 , loss =  2.294319826787813
Epoch  1 , loss =  2.19673604276496
Epoch  1 , loss =  1.8909364544041096
Epoch  1 , loss =  1.7203832686862652
Epoch  1 , loss =  1.6055224000149175
Epoch  1 , loss =  1.5049649229891697
Epoch  1 , loss =  1.4192280562562356
Epoch  1 , loss =  1.3415547056756414
Epoch  1 , loss =  1.2797027842325808
Epoch  1 , loss =  1.212885246424437
Epoch  1 , loss =  1.1562833506435213
Epoch  1 , loss =  1.1094985993260844
Epoch  1 , loss =  1.0602683343579582
Epoch  1 , loss =  1.0184231218229443
Epoch  1 , loss =  0.9824947948547906


In [ ]:
model_2 = Model_CNN_with_g()\n",
model_2 = model.to(dev)\n",
optimizer = torch.optim.SGD(model_2.parameters(), lr=1e-3)

model_trainer_2 = Trainer(model_2,optimizer,loss_fn, print_loss = True,keep_hist = True)
model_trainer_2.fit_x(train_dataloader,30)
model_trainer_2.fit_g(train_dataloader,15)

acc, g, bce = model_metrics(model_2,train_dataloader)
print('Conjunto de treinamento: acc = ', acc, 'média de g = ', g, 'média de bce = ', bce, '\n')
acc, g, bce = model_metrics(model_2,test_dataloader)
print('Conjunto de teste: acc = ', acc, 'média de g = ', g, 'média de bce = ', bce, '\n')

In [ ]:
model_3 = Model_CNN_with_g()
model_3 = model.to(dev)
optimizer = torch.optim.SGD(model_3.parameters(), lr=1e-3)

model_trainer_3 = Trainer(model_3,optimizer,loss_fn, print_loss = True,keep_hist = True)
model_trainer_3.fit_x(train_dataloader,15)
model_trainer_3.fit_g(validation_dataloader,15)

acc, g, bce = model_metrics(model_3,train_dataloader)
print('Conjunto de treinamento: acc = ', acc, 'média de g = ', g, 'média de bce = ', bce, '\n')
acc, g, bce = model_metrics(model_3,test_dataloader)
print('Conjunto de teste: acc = ', acc, 'média de g = ', g, 'média de bce = ', bce, '\n')